In [ ]:
import numpy as np
import pandas as pd
import formulas
import os

In [ ]:
#path to directory where pdb files are stored.
path = r'str'

#making list of files to be analyzed.
files = []
with open('txt file with list of pdb files', 'r') as f:
    for file in f:
        files.append(os.path.join(path, file[:-1]))

In [ ]:
def coordinates1(file):
    """
    Extracting data from pdb as DataFrame.
    
    Args:
        file (str): pdb_file_name.
        
    Returns:
        dfs (list): A list of DataFrames containing coordinates of 
                         desired atoms.
    """
    df = formulas.read_pdb(file)
    SD = df[df['ATOM ID'] == "SD"]
    CG = df[df['ATOM ID'] == "CG"]
    CE = df[df['ATOM ID'] == "CE"]
    O = df[df['Atm'] == "O"]
    N = df[df['Atm'] == "N"]
    dfs = [SD, CG, CE, O, N]
    return dfs

In [ ]:
def coordinates2(file):
    """
    Extracting water molecules data from pdb file as DataFrame.
    
    Args:
        file (str): pdb_file_name.
        
    Returns:
        OH (DataFrame): DataFrame containing water molecule data.
    """
    OH = formulas.read_pdb_water(file)
    return OH

In [ ]:
def find_contacts(dfs):
    """
    Searching for contacts between SD and (O, N, OH).
    
    Args:
        dfs (list): A list of DataFrames containing coordinates of desired atoms.
        
    Returns:
        contacts (list): list with details of atoms making contact with SD.
    """
    SD, CG, CE, O, N, OH = dfs
    contacts = []
    for i1, r1 in SD.iterrows():
        SD_ResNo = [i for i in range((r1['RES INSERT'] - 1), (r1['RES INSERT'] + 2))]
        for i2, r2 in O.iterrows():
            if not r2['RES INSERT'] in SD_ResNo:
                distance = formulas.eu_distance(np.array([r1['X AXIS'], r1['Y AXIS'], r1['Z AXIS']]), 
                                                np.array([r2['X AXIS'], r2['Y AXIS'], r2['Z AXIS']]))
                if distance <= 3.32:
                    contacts.append([r1, r2, distance])
                else:
                    pass
        for i2, r2 in N.iterrows():
            if not r2['RES INSERT'] in SD_ResNo:
                distance = formulas.eu_distance(np.array([r1['X AXIS'], r1['Y AXIS'], r1['Z AXIS']]), 
                                                np.array([r2['X AXIS'], r2['Y AXIS'], r2['Z AXIS']]))
                if distance <= 3.35:
                    contacts.append([r1, r2, distance])
                else:
                    pass
        for i2, r2 in OH.iterrows():
            distance = formulas.eu_distance(np.array([r1['X AXIS'], r1['Y AXIS'], r1['Z AXIS']]), 
                                            np.array([r2['X AXIS'], r2['Y AXIS'], r2['Z AXIS']]))
            if distance <= 3.32:
                contacts.append([r1, r2, distance])
            else:
                pass
    return contacts

In [ ]:
def angles(dfs, contact):
    """
    Searching for contacts between SD and (O, N, OH).
    
    Args:
        dfs (list): A list of DataFrames containing coordinates of desired atoms.
        contact (list): list of details of atom making  contact with SD.
        
    Returns:
        angles_list (list): list of calculated angles (theta-delta) to study directionality
                            of the interaction.
    """
    SD, CG, CE, O, N, OH = dfs
    S = contact[0]
    O = contact[1]
    C1 = CG[(CG["RES SEQ"] == S["RES SEQ"]) & (CG["RES INSERT"] == S["RES INSERT"])]
    C2 = CE[(CE["RES SEQ"] == S["RES SEQ"]) & (CE["RES INSERT"] == S["RES INSERT"])]
    C1 = C1.reset_index(drop=True)
    C2 = C2.reset_index(drop=True)
    p1 = np.array([S['X AXIS'], S['Y AXIS'], S['Z AXIS']])
    p4 = np.array([O['X AXIS'], O['Y AXIS'], O['Z AXIS']])
    p2 = np.array([C1['X AXIS'].iloc[0], C1['Y AXIS'].iloc[0], C1['Z AXIS'].iloc[0]])
    p3 = np.array([C2['X AXIS'].iloc[0], C2['Y AXIS'].iloc[0], C2['Z AXIS'].iloc[0]])
    delta, theta = formulas.theta_phi(p1, p2, p3, p4)
    angles_list = [delta, theta]
    return angles_list

In [ ]:
def seq_run(file):
    """
    Running functions defined in cells before sequentially.
    
    Args:
        file (str): pdb_file_name.
    
    Returns:
        result (DataFrame): A dataframe with details of distance and directional criterion
                             of the residues from beta-turn.
    """
    cols = ['file', 'Met_Chain', 'Met_ResNo', 'IntResChain', 'IntResNo', 'IntAtom', 'IntAtomID',
            'IntRes', 'distance', 'delta', 'theta']
    result = pd.DataFrame(columns=cols)
    dfs = coordinates1(file)
    OH = coordinates2(file)
    dfs.append(OH)
    contacts = find_contacts(dfs)
    for contact in contacts:
        angles_list = angles(dfs, contact)
        contact = contact + angles_list
        res = [file[-8:], contact[0]['RES SEQ'], contact[0]['RES INSERT'], contact[1]['RES SEQ'],
               contact[1]['RES INSERT'], contact[1]['Atm'], contact[1]['ATOM ID'], contact[1]['RESIDUE'],
               contact[2], contact[3], contact[4]]
        result.loc[len(result)] = res
    return result

results = pd.DataFrame()
for file in files:
    result = seq_run(file)
    results = results.append(result, ignore_index=True)
    
results.to_csv('CSC_data.txt', sep='\t', index=False)